##### Imports

In [1]:
import numpy as np
import pandas as pd
import requests
import yahoo_fin.stock_info as si
import math

##### Retrieving stocks

In [2]:
tickers = pd.read_csv('data/sp_500_stocks.csv')
tickers

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


### Retrieving current market data

In [3]:
columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
stocks = pd.DataFrame(columns = columns)
stocks

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [4]:
list(tickers['Ticker'][:5])

['A', 'AAL', 'AAP', 'AAPL', 'ABBV']

In [36]:
for ticker in list(tickers['Ticker']):
    try:
        stock_price = si.get_live_price(ticker)
    except:
        print('Error with ticker: ' + ticker)
        continue
    market_cap = si.get_quote_data(ticker)['marketCap']
    stocks = pd.concat([stocks, pd.DataFrame([[ticker, stock_price, market_cap, 'N/A']], columns = columns)], ignore_index = True)

stocks

Error with ticker: ALXN
Error with ticker: ANTM
Error with ticker: BF.B
Error with ticker: BLL
Error with ticker: BRK.B
Error with ticker: CERN
Error with ticker: COG
Error with ticker: CTL
Error with ticker: CTXS
Error with ticker: CXO
Error with ticker: DISCA
Error with ticker: DISCK
Error with ticker: DRE
Error with ticker: ETFC
Error with ticker: FB
Error with ticker: FBHS
Error with ticker: FLIR
Error with ticker: HFC
Error with ticker: INFO
Error with ticker: KSU
Error with ticker: LB
Error with ticker: MXIM
Error with ticker: MYL
Error with ticker: NBL
Error with ticker: NLOK
Error with ticker: NLSN
Error with ticker: PBCT
Error with ticker: TIF
Error with ticker: VAR
Error with ticker: VIAC
Error with ticker: WLTW
Error with ticker: XLNX


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,143.229996,42376028160,N/A
1,AAL,16.120001,10501067776,N/A
2,AAP,135.130005,8009669120,N/A
3,AAPL,153.830002,2433882783744,N/A
4,ABBV,155.279999,274752438272,N/A
...,...,...,...,...
529,YUM,129.300003,36217966592,N/A
530,ZBH,124.620003,26178177024,N/A
531,ZBRA,305.809998,15720071168,N/A
532,ZION,48.759998,7248124416,N/A


In [41]:
# drop duplicates
stocks = stocks.drop_duplicates()
stocks

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,143.229996,42376028160,N/A
1,AAL,16.120001,10501067776,N/A
2,AAP,135.130005,8009669120,N/A
3,AAPL,153.830002,2433882783744,N/A
4,ABBV,155.279999,274752438272,N/A
...,...,...,...,...
529,YUM,129.300003,36217966592,N/A
530,ZBH,124.620003,26178177024,N/A
531,ZBRA,305.809998,15720071168,N/A
532,ZION,48.759998,7248124416,N/A


In [42]:
# save stocks as csv
stocks.to_csv('data/stocks.csv', index = False)

### Calculating the number of shares to buy

In [25]:
stocks = pd.read_csv('data/stocks.csv')
stocks

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,143.229996,42376028160,NaN
1,AAL,16.120001,10501067776,NaN
2,AAP,135.130005,8009669120,NaN
3,AAPL,153.830002,2433882783744,NaN
4,ABBV,155.279999,274752438272,NaN
...,...,...,...,...
468,YUM,129.300003,36217966592,NaN
469,ZBH,124.620003,26178177024,NaN
470,ZBRA,305.809998,15720071168,NaN
471,ZION,48.759998,7248124416,NaN


In [26]:
# remove NaN values
stocks = stocks[stocks['Stock Price'] > 0]
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 471 entries, 0 to 472
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Ticker                   471 non-null    object 
 1   Stock Price              471 non-null    float64
 2   Market Capitalization    471 non-null    int64  
 3   Number of Shares to Buy  0 non-null      float64
dtypes: float64(2), int64(1), object(1)
memory usage: 18.4+ KB


##### Calcalating the postion size

In [27]:
PORTFOLIO_SIZE = 1000000

In [28]:
position_size = PORTFOLIO_SIZE / len(stocks)
print(position_size)

2123.1422505307855


##### Calculating number of stocks for each ticker

In [37]:
print(stocks.loc[410:430, :])

    Ticker  Stock Price  Market Capitalization  Number of Shares to Buy
410    TJX    78.250000            90417881088                     27.0
411    TMO   558.520020           215270375424                      3.0
412   TMUS   143.720001           175249293312                     14.0
413    TPR    43.540001            10278749184                     48.0
414   TROW   114.589996            25713881088                     18.0
415    TRV   183.350006            42554437632                     11.0
416   TSCO   228.690002            25172594688                      9.0
417    TSN    58.750000            20893026304                     36.0
418     TT   192.059998            43996143616                     11.0
419   TTWO   115.709999            19517384704                     18.0
421    TXN   173.050003           156818948096                     12.0
422    TXT    73.550003            15093710848                     28.0
423    TYL   320.239990            13392211968                  

In [43]:
# set number of shares to buy
stocks.loc[:, 'Number of Shares to Buy'] = np.floor(position_size / stocks.loc[:, 'Stock Price'])
# we round down to ensure we do not go over the portfolio size

stocks.head()

C:\Users\asus\AppData\Local\Temp\ipykernel_20164\3773584864.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks.loc[:, 'Number of Shares to Buy'] = np.floor(position_size / stocks.loc[:, 'Stock Price'])


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,143.229996,42376028160,14.0
1,AAL,16.120001,10501067776,131.0
2,AAP,135.130005,8009669120,15.0
3,AAPL,153.830002,2433882783744,13.0
4,ABBV,155.279999,274752438272,13.0
